# Day 18
## Part 1
This seems so simple that I'm nervous part 2.

In [4]:
def parse_data(s):
    return {tuple(int(x) for x in line.split(',')) 
            for line in s.strip().splitlines()}

test_string = """
2,2,2
1,2,2
3,2,2
2,1,2
2,3,2
2,2,1
2,2,3
2,2,4
2,2,6
1,2,5
3,2,5
2,1,5
2,3,5
"""

test_data = parse_data(test_string)
test_data

{(1, 2, 2),
 (1, 2, 5),
 (2, 1, 2),
 (2, 1, 5),
 (2, 2, 1),
 (2, 2, 2),
 (2, 2, 3),
 (2, 2, 4),
 (2, 2, 6),
 (2, 3, 2),
 (2, 3, 5),
 (3, 2, 2),
 (3, 2, 5)}

In [6]:
def part_1(data):
    sides = 0
    for x, y, z in data:
        for d in -1, 1:
            sides += sum([
                (x + d, y, z) not in data, 
                (x, y + d, z) not in data,
                (x, y, z + d) not in data
            ])
    return sides

part_1(test_data)

64

In [7]:
data = parse_data(open("input").read())
part_1(data)

4444

## Part 2

Yep. How am I going to do this? I'll have a think during the football.